# 🎸 Bass Transcription Model Training

**Goal**: Train a CRNN model that transcribes bass audio → MIDI with per-string/fret predictions.

**Architecture**: CQT → Conv2D → BiLSTM → onset/frame/velocity heads (4 strings × 24 frets)

**Dataset**: Slakh2100-redux (synthesized bass stems + aligned MIDI, ~145 hours total)

**Why Slakh?**
- Every track has isolated bass audio + perfectly aligned MIDI
- 1,710 deduplicated tracks (redux version avoids train/test leakage)
- Covers diverse bass styles via 187 instrument patches
- Audio synthesized from Lakh MIDI Dataset using professional virtual instruments

**Output**: `best_bass_model.pt` — drop into `backend/models/pretrained/`

---
**Estimated Training Time**:
- A100: ~4-8 hours (50 epochs)
- T4: ~12-24 hours

**Disk Requirements**: ~60 GB (Slakh2100 bass stems + MIDI)

## 0. GPU Check & Setup

In [ ]:
import torch
import os

# GPU check
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU: {gpu_name} ({gpu_mem:.1f} GB)')
else:
    print('WARNING: No GPU detected! Training will be very slow.')
    print('Go to Runtime > Change runtime type > GPU (A100 preferred)')

# Mount Google Drive for checkpoint saving
from google.colab import drive
drive.mount('/content/drive')

# Create output directory on Drive
DRIVE_OUTPUT = '/content/drive/MyDrive/bass_model_results'
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
print(f'Checkpoints will save to: {DRIVE_OUTPUT}')

In [ ]:
# Install dependencies
!pip install -q librosa pretty_midi tqdm soundfile pyyaml

## 1. Download Slakh2100 Bass Stems

Slakh2100 is ~105 GB total. We only need bass stems + MIDI (~15-20 GB).
We'll download the full dataset and extract just bass.

In [ ]:
import subprocess
import shutil
from pathlib import Path

DATA_DIR = Path('/content/slakh_bass')
DATA_DIR.mkdir(exist_ok=True)

SLAKH_ROOT = Path('/content/slakh2100_flac_redux')

# Check if we already have extracted bass data (Colab reconnect)
existing_tracks = list(DATA_DIR.glob('*_Track*'))
if len(existing_tracks) > 100:
    print(f'Found {len(existing_tracks)} existing bass tracks, skipping download')
elif SLAKH_ROOT.exists() and len(list(SLAKH_ROOT.glob('**/metadata.yaml'))) > 100:
    print(f'Slakh already extracted, skipping download (will extract bass in next cell)')
else:
    print('Downloading Slakh2100-redux from Zenodo...')
    print('This is ~97 GB — grab a coffee')
    print()

    TAR_PATH = '/content/slakh2100.tar.gz'

    # Download Slakh2100-redux (deduplicated version)
    !wget -q --show-progress -O {TAR_PATH} \
        'https://zenodo.org/records/4599666/files/slakh2100_flac_redux.tar.gz?download=1'

    # Check disk space before extracting
    import shutil as sh
    total, used, free = sh.disk_usage('/content')
    free_gb = free / 1e9
    print(f'\nDisk space: {free_gb:.1f} GB free')

    if free_gb < 110:
        # Not enough space to keep tar + extracted simultaneously
        # Use streaming approach: extract in pieces, deleting tar progressively
        print('Using streaming extraction to manage disk space...')

        # First, extract ONLY metadata files (tiny) to identify bass stems
        print('Step 1: Extracting metadata files only...')
        !tar xzf {TAR_PATH} -C /content/ --wildcards '*/metadata.yaml' 2>/dev/null || true

        # Now delete the tar to free ~97GB
        tar_file = Path(TAR_PATH)
        if tar_file.exists():
            tar_file.unlink()
            print('Deleted tar.gz to free disk space')

        # Re-download only the bass stems we need using the metadata
        # First, scan metadata to find bass stem IDs
        print('Step 2: Scanning metadata for bass stems...')
        import yaml
        bass_files_needed = []
        for meta_path in sorted(SLAKH_ROOT.glob('**/metadata.yaml')):
            try:
                with open(meta_path) as f:
                    meta = yaml.safe_load(f)
                track_dir = meta_path.parent
                rel_track = track_dir.relative_to(Path('/content'))
                for stem_id, stem_info in meta.get('stems', {}).items():
                    prog = stem_info.get('program_num', -1)
                    is_drum = stem_info.get('is_drum', False)
                    if not is_drum and prog in range(32, 40):
                        bass_files_needed.append(f'{rel_track}/stems/{stem_id}.flac')
                        bass_files_needed.append(f'{rel_track}/MIDI/{stem_id}.mid')
            except Exception:
                continue

        print(f'Found {len(bass_files_needed) // 2} bass stems to extract')

        # Write file list and re-download + extract only those files
        with open('/content/bass_files.txt', 'w') as f:
            for fn in bass_files_needed:
                f.write(fn + '\n')

        print('Step 3: Re-downloading and extracting only bass stems...')
        !wget -q --show-progress -O - \
            'https://zenodo.org/records/4599666/files/slakh2100_flac_redux.tar.gz?download=1' | \
            tar xzf - -C /content/ -T /content/bass_files.txt 2>/dev/null || true

        print('Done with streaming extraction')
    else:
        # Enough disk space for normal extraction
        print('Extracting full dataset...')
        !tar xzf {TAR_PATH} -C /content/ 2>/dev/null || true

        # Delete tar immediately to free space
        tar_file = Path(TAR_PATH)
        if tar_file.exists():
            tar_file.unlink()
            print('Deleted tar.gz to free disk space')

    print('Extraction complete')

In [ ]:
import yaml
import shutil
from pathlib import Path

# Extract bass stems from Slakh2100
# Slakh metadata.yaml tells us which stem number is bass

SLAKH_ROOT = Path('/content/slakh2100_flac_redux')
DATA_DIR = Path('/content/slakh_bass')
DATA_DIR.mkdir(exist_ok=True)

# Bass MIDI programs: 32-39 (Acoustic Bass, Electric Bass, etc.)
BASS_PROGRAMS = set(range(32, 40))

bass_tracks = []
skipped = 0

for split in ['train', 'validation', 'test']:
    split_dir = SLAKH_ROOT / split
    if not split_dir.exists():
        print(f'Split {split} not found, skipping')
        continue

    for track_dir in sorted(split_dir.iterdir()):
        if not track_dir.is_dir():
            continue

        metadata_file = track_dir / 'metadata.yaml'
        if not metadata_file.exists():
            skipped += 1
            continue

        with open(metadata_file) as f:
            meta = yaml.safe_load(f)

        # Find bass stem(s)
        stems_meta = meta.get('stems', {})
        for stem_id, stem_info in stems_meta.items():
            program = stem_info.get('program_num', -1)
            is_drum = stem_info.get('is_drum', False)

            if is_drum or program not in BASS_PROGRAMS:
                continue

            # Found a bass stem!
            audio_file = track_dir / 'stems' / f'{stem_id}.flac'
            midi_file = track_dir / 'MIDI' / f'{stem_id}.mid'

            if audio_file.exists() and midi_file.exists():
                # Copy to our bass directory
                track_name = f"{split}_{track_dir.name}_{stem_id}"
                out_dir = DATA_DIR / track_name
                out_dir.mkdir(exist_ok=True)

                shutil.copy2(audio_file, out_dir / 'bass.flac')
                shutil.copy2(midi_file, out_dir / 'bass.mid')

                bass_tracks.append({
                    'track': track_name,
                    'split': split,
                    'program': program,
                    'audio': str(out_dir / 'bass.flac'),
                    'midi': str(out_dir / 'bass.mid'),
                })

print(f'\n✅ Extracted {len(bass_tracks)} bass stems')
print(f'   Skipped {skipped} tracks (no metadata)')
print(f'   Train: {sum(1 for t in bass_tracks if t["split"] == "train")}')
print(f'   Val: {sum(1 for t in bass_tracks if t["split"] == "validation")}')
print(f'   Test: {sum(1 for t in bass_tracks if t["split"] == "test")}')

# Always clean up the full Slakh to save disk — we only need the extracted bass stems
if SLAKH_ROOT.exists():
    print('\nCleaning up full Slakh archive to save disk space...')
    shutil.rmtree(SLAKH_ROOT, ignore_errors=True)
    print('✅ Freed disk space')

slakh_tar = Path('/content/slakh2100.tar.gz')
if slakh_tar.exists():
    slakh_tar.unlink()
    print('✅ Removed tar.gz')

# Verify we got enough data
if len(bass_tracks) < 100:
    print(f'\n⚠️ WARNING: Only found {len(bass_tracks)} bass tracks.')
    print('Expected ~1000+. Extraction may have been incomplete.')
    print('Check disk space and re-run if needed.')

## 2. Dataset & Preprocessing

In [ ]:
# ============================================================================
# CONFIG — must match inference in bass_transcriber.py EXACTLY
# ============================================================================

CONFIG = {
    'sample_rate': 22050,
    'hop_length': 256,
    'n_bins': 84,           # CQT bins (7 octaves × 12 bins/octave)
    'bins_per_octave': 12,
    'chunk_duration': 5.0,  # seconds per training chunk
    'num_strings': 4,
    'num_frets': 24,
    'tuning': [28, 33, 38, 43],  # E1, A1, D2, G2 (4-string bass, standard)
    'batch_size': 16,
    'learning_rate': 1e-4,
    'num_epochs': 50,
}

SAMPLE_RATE = CONFIG['sample_rate']
HOP_LENGTH = CONFIG['hop_length']
N_BINS = CONFIG['n_bins']
BINS_PER_OCTAVE = CONFIG['bins_per_octave']
CHUNK_DURATION = CONFIG['chunk_duration']
NUM_STRINGS = CONFIG['num_strings']
NUM_FRETS = CONFIG['num_frets']
TUNING = CONFIG['tuning']   # MIDI notes for open strings
CHUNK_SAMPLES = int(CHUNK_DURATION * SAMPLE_RATE)  # 110250
CHUNK_FRAMES = int(CHUNK_SAMPLES / HOP_LENGTH)      # 430

print(f'Chunk: {CHUNK_DURATION}s = {CHUNK_SAMPLES} samples = {CHUNK_FRAMES} frames')
print(f'Bass tuning (MIDI): {TUNING} = E1, A1, D2, G2')
print(f'Output shape per chunk: ({NUM_STRINGS}, {NUM_FRETS}, {CHUNK_FRAMES})')
print(f'Total output neurons: {NUM_STRINGS * NUM_FRETS} = {NUM_STRINGS}×{NUM_FRETS}')

In [ ]:
import numpy as np
import librosa
import pretty_midi
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


def midi_note_to_string_fret(midi_note, tuning=TUNING, num_frets=NUM_FRETS):
    """
    Map a MIDI note to the best (string, fret) position on bass.

    Prefers lower positions (closer to nut) and lower strings.
    Returns None if note is out of range.
    """
    candidates = []
    for s, open_note in enumerate(tuning):
        fret = midi_note - open_note
        if 0 <= fret < num_frets:
            # Prefer lower fret positions, then lower strings
            candidates.append((s, fret, fret + s * 0.1))

    if not candidates:
        return None

    # Sort by preference score (lower = better)
    candidates.sort(key=lambda x: x[2])
    return candidates[0][0], candidates[0][1]


def midi_to_targets(midi_path, total_frames, sr=SAMPLE_RATE, hop=HOP_LENGTH):
    """
    Convert MIDI file to onset/frame/velocity target tensors.

    Returns:
        onset_target: (num_strings, num_frets, total_frames) binary
        frame_target: (num_strings, num_frets, total_frames) binary
        velocity_target: (num_strings, num_frets, total_frames) float [0-1]
    """
    onset = np.zeros((NUM_STRINGS, NUM_FRETS, total_frames), dtype=np.float32)
    frame = np.zeros((NUM_STRINGS, NUM_FRETS, total_frames), dtype=np.float32)
    velocity = np.zeros((NUM_STRINGS, NUM_FRETS, total_frames), dtype=np.float32)

    frames_per_sec = sr / hop

    try:
        midi = pretty_midi.PrettyMIDI(str(midi_path))
    except Exception:
        return onset, frame, velocity

    for instrument in midi.instruments:
        if instrument.is_drum:
            continue

        for note in instrument.notes:
            result = midi_note_to_string_fret(note.pitch)
            if result is None:
                continue

            s, f = result
            onset_frame = int(note.start * frames_per_sec)
            offset_frame = int(note.end * frames_per_sec)

            if onset_frame >= total_frames:
                continue

            offset_frame = min(offset_frame, total_frames - 1)
            vel = note.velocity / 127.0

            # Onset: single frame at note start
            onset[s, f, onset_frame] = 1.0

            # Frame: all frames while note is active
            frame[s, f, onset_frame:offset_frame + 1] = 1.0

            # Velocity: at onset frame
            velocity[s, f, onset_frame] = vel

    return onset, frame, velocity


class BassTranscriptionDataset(Dataset):
    """
    Dataset that yields (cqt_chunk, onset_target, frame_target, velocity_target)
    from paired bass audio + MIDI files.
    """

    def __init__(self, track_list, chunks_per_track=8):
        """
        Args:
            track_list: List of dicts with 'audio' and 'midi' paths
            chunks_per_track: Random chunks to sample per track per epoch
        """
        self.tracks = track_list
        self.chunks_per_track = chunks_per_track
        self._cache = {}  # Cache loaded audio/MIDI to avoid re-reading

    def __len__(self):
        return len(self.tracks) * self.chunks_per_track

    def _load_track(self, idx):
        track_idx = idx // self.chunks_per_track

        if track_idx in self._cache:
            return self._cache[track_idx]

        track = self.tracks[track_idx]

        # Load audio
        try:
            audio, sr = librosa.load(track['audio'], sr=SAMPLE_RATE, mono=True)
        except Exception:
            audio = np.zeros(CHUNK_SAMPLES * 2, dtype=np.float32)

        # Compute full CQT
        cqt = np.abs(librosa.cqt(
            audio, sr=SAMPLE_RATE,
            hop_length=HOP_LENGTH,
            n_bins=N_BINS,
            bins_per_octave=BINS_PER_OCTAVE
        ))
        cqt = np.log(cqt + 1e-8).astype(np.float32)  # MUST match inference!

        total_frames = cqt.shape[1]

        # Build targets from MIDI
        onset, frame, velocity = midi_to_targets(track['midi'], total_frames)

        result = (cqt, onset, frame, velocity, total_frames)

        # Cache (limit cache size to avoid OOM)
        if len(self._cache) < 200:
            self._cache[track_idx] = result

        return result

    def __getitem__(self, idx):
        cqt, onset, frame, velocity, total_frames = self._load_track(idx)

        # Random chunk
        if total_frames <= CHUNK_FRAMES:
            # Pad short audio
            pad_frames = CHUNK_FRAMES - total_frames
            cqt_chunk = np.pad(cqt, ((0, 0), (0, pad_frames)))
            onset_chunk = np.pad(onset, ((0, 0), (0, 0), (0, pad_frames)))
            frame_chunk = np.pad(frame, ((0, 0), (0, 0), (0, pad_frames)))
            vel_chunk = np.pad(velocity, ((0, 0), (0, 0), (0, pad_frames)))
        else:
            start = np.random.randint(0, total_frames - CHUNK_FRAMES)
            cqt_chunk = cqt[:, start:start + CHUNK_FRAMES]
            onset_chunk = onset[:, :, start:start + CHUNK_FRAMES]
            frame_chunk = frame[:, :, start:start + CHUNK_FRAMES]
            vel_chunk = velocity[:, :, start:start + CHUNK_FRAMES]

        return (
            torch.from_numpy(cqt_chunk),
            torch.from_numpy(onset_chunk),
            torch.from_numpy(frame_chunk),
            torch.from_numpy(vel_chunk),
        )


print('✅ Dataset class defined')

In [ ]:
# Build train/val splits
import random

# Use Slakh's built-in splits
train_tracks = [t for t in bass_tracks if t['split'] == 'train']
val_tracks = [t for t in bass_tracks if t['split'] in ('validation', 'test')]

# If we didn't get proper splits (e.g., manual extraction), do 90/10
if len(val_tracks) < 10:
    random.shuffle(bass_tracks)
    split_idx = int(len(bass_tracks) * 0.9)
    train_tracks = bass_tracks[:split_idx]
    val_tracks = bass_tracks[split_idx:]

print(f'Train: {len(train_tracks)} tracks')
print(f'Val:   {len(val_tracks)} tracks')

train_dataset = BassTranscriptionDataset(train_tracks, chunks_per_track=8)
val_dataset = BassTranscriptionDataset(val_tracks, chunks_per_track=4)

train_loader = DataLoader(
    train_dataset, batch_size=CONFIG['batch_size'],
    shuffle=True, num_workers=2, pin_memory=True,
    drop_last=True
)
val_loader = DataLoader(
    val_dataset, batch_size=CONFIG['batch_size'],
    shuffle=False, num_workers=2, pin_memory=True
)

print(f'Train batches/epoch: {len(train_loader)}')
print(f'Val batches/epoch:   {len(val_loader)}')

# Sanity check: load one batch
cqt_b, onset_b, frame_b, vel_b = next(iter(train_loader))
print(f'\nBatch shapes:')
print(f'  CQT:      {cqt_b.shape}')      # (B, 84, 430)
print(f'  Onset:    {onset_b.shape}')     # (B, 4, 24, 430)
print(f'  Frame:    {frame_b.shape}')     # (B, 4, 24, 430)
print(f'  Velocity: {vel_b.shape}')       # (B, 4, 24, 430)
print(f'  Onset active: {onset_b.sum():.0f} / {onset_b.numel()}')
print(f'  Frame active: {frame_b.sum():.0f} / {frame_b.numel()}')

## 3. Model Architecture

In [ ]:
import torch
import torch.nn as nn


class BassTranscriptionModel(nn.Module):
    """
    CRNN for bass transcription: CQT → Conv2D → BiLSTM → onset/frame/velocity

    Architecture follows our proven drum/guitar tab pattern:
    - Conv2D stack pools frequency axis only (preserves time)
    - BiLSTM captures temporal dependencies
    - Separate onset, frame, and velocity prediction heads
    - Frame prediction conditioned on onset (OaF-style)

    Input:  (batch, n_bins=84, time)
    Output: onset  (batch, 4, 24, time)  — string × fret binary onset
            frame  (batch, 4, 24, time)  — string × fret binary frame
            velocity (batch, 4, 24, time) — velocity at onset points
    """

    def __init__(self, n_bins=84, num_strings=4, num_frets=24,
                 hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()

        self.num_strings = num_strings
        self.num_frets = num_frets
        output_size = num_strings * num_frets  # 96

        # CNN encoder — pools frequency only via MaxPool2d((2,1))
        self.cnn = nn.Sequential(
            # Block 1: 84 → 42
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # freq only: 84 → 42
            nn.Dropout2d(dropout),

            # Block 2: 42 → 21
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # freq only: 42 → 21
            nn.Dropout2d(dropout),

            # Block 3: 21 → 10
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # freq only: 21 → 10
            nn.Dropout2d(dropout),
        )

        # After CNN: (batch, 128, 10, time)
        cnn_output_dim = 128 * 10  # 1280

        # Onset LSTM
        self.onset_lstm = nn.LSTM(
            input_size=cnn_output_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
        )

        # Frame LSTM (conditioned on onset predictions)
        self.frame_lstm = nn.LSTM(
            input_size=cnn_output_dim + output_size,  # CNN + onset_pred
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
        )

        lstm_output_dim = hidden_size * 2  # bidirectional → 512

        # Output heads
        self.onset_head = nn.Linear(lstm_output_dim, output_size)
        self.frame_head = nn.Linear(lstm_output_dim, output_size)
        self.velocity_head = nn.Sequential(
            nn.Linear(lstm_output_dim, output_size),
            nn.Sigmoid(),  # velocity is 0-1
        )

    def forward(self, x):
        """
        Args:
            x: (batch, n_bins, time) — CQT spectrogram
        Returns:
            onset:    (batch, num_strings, num_frets, time)
            frame:    (batch, num_strings, num_frets, time)
            velocity: (batch, num_strings, num_frets, time)
        """
        batch, n_bins, time = x.shape

        # CNN: (batch, 1, n_bins, time) → (batch, 128, 10, time)
        cnn_out = self.cnn(x.unsqueeze(1))

        # Reshape for LSTM: (batch, time, features)
        cnn_features = cnn_out.permute(0, 3, 1, 2)  # (B, T, 128, 10)
        cnn_features = cnn_features.reshape(batch, time, -1)  # (B, T, 1280)

        # Onset prediction
        onset_out, _ = self.onset_lstm(cnn_features)  # (B, T, 512)
        onset_logits = self.onset_head(onset_out)     # (B, T, 96)
        onset_pred = torch.sigmoid(onset_logits)

        # Frame prediction (conditioned on onset)
        frame_input = torch.cat([cnn_features, onset_pred.detach()], dim=-1)
        frame_out, _ = self.frame_lstm(frame_input)   # (B, T, 512)
        frame_logits = self.frame_head(frame_out)     # (B, T, 96)
        frame_pred = torch.sigmoid(frame_logits)

        # Velocity prediction (from onset LSTM features)
        velocity_pred = self.velocity_head(onset_out)  # (B, T, 96)

        # Reshape to (batch, strings, frets, time)
        def reshape_output(t):
            return t.permute(0, 2, 1).reshape(
                batch, self.num_strings, self.num_frets, time
            )

        return reshape_output(onset_pred), reshape_output(frame_pred), reshape_output(velocity_pred)


# Test model
model = BassTranscriptionModel()
total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameters: {total_params:,}')

# Dummy forward pass
dummy = torch.randn(2, N_BINS, CHUNK_FRAMES)
onset, frame, vel = model(dummy)
print(f'Input:    {dummy.shape}')
print(f'Onset:    {onset.shape}')    # (2, 4, 24, 430)
print(f'Frame:    {frame.shape}')    # (2, 4, 24, 430)
print(f'Velocity: {vel.shape}')      # (2, 4, 24, 430)
assert onset.shape == (2, NUM_STRINGS, NUM_FRETS, CHUNK_FRAMES)
print('\n✅ Model architecture verified')

## 4. Training Loop

In [ ]:
import time
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training on: {device}')

model = BassTranscriptionModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=5, factor=0.5
)

onset_loss_fn = nn.BCELoss()
frame_loss_fn = nn.BCELoss()
velocity_loss_fn = nn.MSELoss()

# Resume from checkpoint if available
start_epoch = 0
best_val_loss = float('inf')

# Check for existing checkpoints on Drive
checkpoint_files = sorted(Path(DRIVE_OUTPUT).glob('checkpoint_epoch_*.pt'))
best_model_path = Path(DRIVE_OUTPUT) / 'best_bass_model.pt'

if checkpoint_files:
    latest = checkpoint_files[-1]
    print(f'Resuming from {latest.name}...')
    ckpt = torch.load(latest, map_location=device, weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_val_loss = ckpt.get('val_loss', float('inf'))
    print(f'Resumed at epoch {start_epoch}, best_val_loss={best_val_loss:.5f}')
elif best_model_path.exists():
    print(f'Found best model, resuming...')
    ckpt = torch.load(best_model_path, map_location=device, weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_val_loss = ckpt.get('val_loss', float('inf'))
    print(f'Resumed at epoch {start_epoch}, best_val_loss={best_val_loss:.5f}')

print(f'\nStarting training from epoch {start_epoch}...')
print(f'Epochs: {CONFIG["num_epochs"]}, Batch size: {CONFIG["batch_size"]}')
print(f'LR: {CONFIG["learning_rate"]}')

In [ ]:
# ============================================================================
# TRAINING LOOP
# ============================================================================

NUM_EPOCHS = CONFIG['num_epochs']
train_losses = []
val_losses = []

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start = time.time()

    # ---- Train ----
    model.train()
    running_loss = 0.0
    num_batches = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{NUM_EPOCHS}')
    for cqt_batch, onset_tgt, frame_tgt, vel_tgt in pbar:
        cqt_batch = cqt_batch.to(device)
        onset_tgt = onset_tgt.to(device)
        frame_tgt = frame_tgt.to(device)
        vel_tgt = vel_tgt.to(device)

        # Forward pass
        onset_pred, frame_pred, vel_pred = model(cqt_batch)

        # Compute loss
        loss_onset = onset_loss_fn(onset_pred, onset_tgt)
        loss_frame = frame_loss_fn(frame_pred, frame_tgt)

        # Velocity loss only where onsets exist
        onset_mask = onset_tgt > 0.5
        if onset_mask.any():
            loss_vel = velocity_loss_fn(
                vel_pred[onset_mask], vel_tgt[onset_mask]
            )
        else:
            loss_vel = torch.tensor(0.0, device=device)

        loss = loss_onset + loss_frame + 0.5 * loss_vel

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1
        pbar.set_postfix(loss=f'{loss.item():.4f}')

    train_loss = running_loss / max(num_batches, 1)
    train_losses.append(train_loss)

    # ---- Validation ----
    model.eval()
    val_running = 0.0
    val_batches = 0

    with torch.no_grad():
        for cqt_batch, onset_tgt, frame_tgt, vel_tgt in val_loader:
            cqt_batch = cqt_batch.to(device)
            onset_tgt = onset_tgt.to(device)
            frame_tgt = frame_tgt.to(device)
            vel_tgt = vel_tgt.to(device)

            onset_pred, frame_pred, vel_pred = model(cqt_batch)

            loss_onset = onset_loss_fn(onset_pred, onset_tgt)
            loss_frame = frame_loss_fn(frame_pred, frame_tgt)

            onset_mask = onset_tgt > 0.5
            if onset_mask.any():
                loss_vel = velocity_loss_fn(
                    vel_pred[onset_mask], vel_tgt[onset_mask]
                )
            else:
                loss_vel = torch.tensor(0.0, device=device)

            loss = loss_onset + loss_frame + 0.5 * loss_vel
            val_running += loss.item()
            val_batches += 1

    val_loss = val_running / max(val_batches, 1)
    val_losses.append(val_loss)

    # LR scheduler
    scheduler.step(val_loss)

    elapsed = time.time() - epoch_start
    lr = optimizer.param_groups[0]['lr']

    print(f'Epoch {epoch}/{NUM_EPOCHS} | '
          f'Train: {train_loss:.4f} | Val: {val_loss:.4f} | '
          f'LR: {lr:.2e} | Time: {elapsed:.0f}s')

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'config': CONFIG,
        }, str(best_model_path))
        print(f'  ✅ Best model saved (val_loss={val_loss:.5f})')

    # Periodic checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        ckpt_path = Path(DRIVE_OUTPUT) / f'checkpoint_epoch_{epoch:03d}.pt'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'config': CONFIG,
        }, str(ckpt_path))
        print(f'  💾 Checkpoint saved: {ckpt_path.name}')

print(f'\n🎉 Training complete! Best val_loss: {best_val_loss:.5f}')
print(f'Best model saved to: {best_model_path}')

## 5. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Train Loss', color='steelblue')
ax.plot(val_losses, label='Val Loss', color='coral')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title(f'Bass Transcription Training (best val_loss={best_val_loss:.5f})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{DRIVE_OUTPUT}/training_curves.png', dpi=150)
plt.show()

## 6. Evaluation

In [ ]:
# Load best model and evaluate
ckpt = torch.load(str(best_model_path), map_location=device, weights_only=False)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

print(f'Loaded best model from epoch {ckpt["epoch"]}, val_loss={ckpt["val_loss"]:.5f}')
print(f'Config: {ckpt["config"]}')

# Quick inference test
with torch.no_grad():
    test_input = torch.randn(1, N_BINS, CHUNK_FRAMES).to(device)
    onset, frame, vel = model(test_input)
    print(f'\nInference test:')
    print(f'  Input:    {test_input.shape}')
    print(f'  Onset:    {onset.shape}, range [{onset.min():.3f}, {onset.max():.3f}]')
    print(f'  Frame:    {frame.shape}, range [{frame.min():.3f}, {frame.max():.3f}]')
    print(f'  Velocity: {vel.shape}, range [{vel.min():.3f}, {vel.max():.3f}]')

print(f'\n✅ Model ready for deployment!')
print(f'Copy {best_model_path} to backend/models/pretrained/best_bass_model.pt')